<a href="https://colab.research.google.com/github/AeroPace/ME-592/blob/main/PointNet_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PointNet Google Colab Wrapper

The PointNet source code https://github.com/charlesq34/pointnet has become out of date to recent Python, Tensorflow, and Google Colab updates. This Google Colab .ipynb session acts as a wrapper to execute either the ModelNet40 or ShapeNet datasets on the original PointNet model without any compromises to the algorithm or developers intent.

Updates were made to the source code to handle out-dated web-page certificates to retrieve the datasets as well as handling depreciated versions of Tensorflow and depreciated Python 2 syntax handling to execute in Python 3.7 environments. Source-Code updates were made and saved off at https://github.com/AeroPace/ME-592/blob/main/HW4/pointnet-masterv2.zip and called directly into memory below.

##Google Colab Session Setup

Check whether there are pre-existing processes running on the GPU

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 16 21:39:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@PURPOSE: Upload Pointnet Source code compatible to Google Colab Session
#          with Python 3.7 and updated data certificates

# Upload and unzip the pointnet-masterv2.zip file into the Google Colab session
!wget --no-check-certificate https://github.com/AeroPace/ME-592/raw/main/HW4/pointnet-masterv2.zip
!unzip pointnet-masterv2.zip

# Remove the zip file from the Google Colab sesion
!rm pointnet-masterv2.zip

# Moving Directories to local /content directories
!mv pointnet-master/doc /content/doc
!mv pointnet-master/data /content/data
!mv pointnet-master/utils /content/utils
!mv pointnet-master/models /content/models
!mv pointnet-master/part_seg /content/part_seg
!mv pointnet-master/sem_seg /content/sem_seg

# Move files to local /content directory
!mv pointnet-master/LICENSE /content/
!mv pointnet-master/README.md /content/
!mv pointnet-master/evaluate.py /content/
!mv pointnet-master/provider.py /content/
!mv pointnet-master/train.py /content/

# Installing the last available depreciated version of tensorflow, v1.15.0, that
# can stabely run the PointNet source code with Google Colab
!pip install tensorflow===1.15.0
import tensorflow as tf
tf.__version__

##Running PointNet on the ShapeNet dataset

In [ ]:
#@PURPOSE: Download the ShapeNet datasets organize the directories

!sh part_seg/download_data.sh
!mv hdf5_data /content/part_seg/
!mv PartAnnotation /content/part_seg/

In [6]:
#@PURPOSE: Train and evaluate the PointNet model on the ShapeNet dataset

''' Hyperparameters run-options :
'--gpu', type=int, default=0, help='GPU to use [default: GPU 0]'
'--batch', type=int, default=32, help='Batch Size during training [default: 32]'
'--epoch', type=int, default=200, help='Epoch to run [default: 50]'
'--point_num', type=int, default=2048, help='Point Number [256/512/1024/2048]'
'--output_dir', type=str, default='train_results', help='Directory that stores all training logs and trained models'
'--wd', type=float, default=0, help='Weight Decay [Default: 0.0]'
'''

# Train the PointNet model on the ShapeNet Dataset
!python part_seg/train.py \
    --gpu 0 \
    --batch 32 \
    --epoch 10 \
    --point_num 2048 \
    --output_dir train_results \
    --wd 0.2

# Evaluate the accuracy of the PointNet model on the ShapeNet verf. dataset
!python part_seg/test.py

Instructions for updating:
non-resource variables are not supported in the long term
#### Batch Size: 32
#### Point Number: 2048
#### Training using GPU: 0
### Training epoch: 10

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2022-04-16 22:16:47.048544: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-16 22:16:47.053393: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-04-16 22:16:47.053593: I tensorflow/compiler

##Running PointNet on the ModelNet40 dataset

In [ ]:
#@PURPOSE: Train and evaluate the PointNet model on the ModelNet40 dataset

''' Hyperparameters run-options :
'--gpu', type=int, default=0, help='GPU to use [default: GPU 0]'
'--model', default='pointnet_cls', help='Model name: pointnet_cls or pointnet_cls_basic [default: pointnet_cls]'
'--log_dir', default='log', help='Log dir [default: log]'
'--num_point', type=int, default=1024, help='Point Number [256/512/1024/2048] [default: 1024]'
'--max_epoch', type=int, default=250, help='Epoch to run [default: 250]'
'--batch_size', type=int, default=32, help='Batch Size during training [default: 32]'
'--learning_rate', type=float, default=0.001, help='Initial learning rate [default: 0.001]'
'--momentum', type=float, default=0.9, help='Initial learning rate [default: 0.9]'
'--optimizer', default='adam', help='adam or momentum [default: adam]'
'--decay_step', type=int, default=200000, help='Decay step for lr decay [default: 200000]'
'--decay_rate', type=float, default=0.7, help='Decay rate for lr decay [default: 0.8]'
'''
# Train the PointNet model on the ModelNet40 Dataset
!python train.py \
    --gpu 0 \
    --model pointnet_cls \
    --max_epoch 1 \
    --log_dir log \
    --num_point 1024 \
    --max_epoch 1 \
    --batch_size 32 \
    --learning_rate 0.001 \
    --momentum 0.9 \
    --optimizer adam \
    --decay_step 200000 \
    --decay_rate 0.7

# Evaluate the accuracy of the PointNet model on the ModelNet40 verf. dataset
!python evaluate.py